The data used in this notebook were obtained using the code in /common/get_Spotify_audio_features.ipynb

A description of the audio features is documented at https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [0]:
# Suppress output of this cell
%%capture

"""Install and import dependencies."""
# Import pandas for dataframes
import pandas as pd 

# Import matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline 

# Import numpy to help with plots
import numpy as np

# Import scikit-learn for scaling and k-means clustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [3]:
"""Set up connection to Google Drive."""

# import Drive helper and mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
"""Define paths and load file audio features"""

# Main path in Google Drive
home_path = '/content/drive/My Drive/Colab Notebooks'

# Path to save data
data_path = f'{home_path}/Data/Spotify audio features'

# File name
save_file_name = 'oku_hanako_audio_features.csv'

# Load file
all_songs_features = pd.read_csv(f'{data_path}/{save_file_name}')

all_songs_features.head(5)

,album,track,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Kasumisou,Kaban no Naka no Yakimochi,3krX7tJSZtgcExvoz2Brdt,0.541,0.535,329067,0.323,0.000002,5,0.113,-7.980,1,0.0268,78.248,4,0.339
1,Kasumisou,Hontowane,2iU639pRsHohsb7JnD3O7s,0.343,0.378,272933,0.473,0.000003,5,0.180,-8.124,1,0.0258,175.899,4,0.572
2,Kasumisou,Aenakutemo,3brkLSKOt1XqeNWs26q803,0.926,0.361,314493,0.207,0.000699,11,0.128,-12.718,1,0.0290,81.101,4,0.335
3,Kasumisou,Zettai,2zOZ64scyLxDLCfWMVzBcC,0.850,0.406,303000,0.297,0.000196,9,0.183,-10.697,0,0.0257,83.399,4,0.171
4,Kasumisou,Negai,76HWQNnmamYdndbKeB5aSo,0.843,0.455,332507,0.393,0.003550,1,0.231,-7.520,1,0.0275,79.010,4,0.220


In [0]:


"""
1. dedupe
-- https://github.com/JMPerez/spotify-dedup/blob/master/app/scripts/deduplicator.js

2. feature selection
-- 

3. feature engineering -- encode categorical variables, e.g. variable for A, B, C etc
-- 

4. clustering k-means with number of clusters evaluation
-- https://www.kdnuggets.com/2019/10/clustering-metrics-better-elbow-method.html

5. PCA and plot

6. Publish results on a Tableau public dashboard for others to play with
"""